## T-GCN

### Algorithm

+ 先理一下, 看看论文和代码
+ 唔, 他这个就等于做完GraphConv之后扔到GRU里去算一个hidden state, 或者说output
+ 那这个hidden的shape是? ok自己指定的
+ 所以我要使用PyG做的话, 应该就是套着来, 写个loop, 里面是conv得到的结果扔到GRU
+ 但是有个问题, GRU默认自己处理多个seq的, 我这种不是等于单步的去处理吗?
+ 不对, 每个seq的conv本身又不依赖hidden state, 所以我就是每个seq都做conv之后, 再把seq作为seq扔到gru里去应该就可以
+ 就是得处理shape的问题, 这个麻烦..
***
+ 仔细看了他的代码, 额, 他是在原本GCN里的`AXW+b`这个式子里, 强行给X加了个h, 变成`A[X+h]W +b`, 然后去先更新一次Conv, 再去做GRU, 手动loop实现seq里h的迭代
+ 额, 和我想的还不太一样那.....
***
+ 我懂了, 他其实是把GRU里hidden state的更新也考虑了连接节点的影响, 就是不是`xW+b`, 而还是`AxW+b`, 额, 话说有必要吗, 如果我这样呢? `((AxW+b)+h)*W2+b2`, 就是我hidden_state不做A左乘, 而是直接拿去做计算? 感觉可以试试, 就当增进理解了...

In [1]:
import os, os.path
import sys
import time

In [2]:
import numpy as np
import pandas as pd

import torch
import torch_geometric as PyG
# from torch.nn import Linear
# import torch.nn as nn
# from torch_geometric.nn import GCNConv

from torch_geometric.data import Data
# from torch_geometric.loader import DataLoader


import matplotlib.pyplot as plt

import sys

sys.path.append('../../ai.rdee')

import ai_rdee

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [19]:
# import torch
from torch_geometric.data import InMemoryDataset


class TGCN_dataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None, pre_filter=None):
        super().__init__(root, transform, pre_transform, pre_filter)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return ['data/sz_adj.csv', 'data/sz_speed.csv']

    @property
    def processed_file_names(self):
        return ['data_sz.pt']

    def download(self):
        # Download to `self.raw_dir`.
        pass
        
    def process(self):
        # Read data into huge `Data` list.
        fn_adj, fn_spd = self.raw_file_names
        df_spd = pd.read_csv(fn_spd)
        S = df_spd.values.T
        df_adj = pd.read_csv(fn_adj, index_col=None, header = None)

        A = df_adj.values
        A.shape

        eis = torch.tensor(np.argwhere(A == 1).T)
        S = torch.Tensor(S)
        data_list = []
        for i in range(S.shape[1]-14):
            data = Data(x=S[:,i:i+12], y=S[:,i+12:i+15], edge_index=eis)
            data_list.append(data)
        
        
        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]

        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [21]:
dsG = TGCN_dataset('./data')

In [22]:
ratio_test = 0.2
size_test = int(ratio_test * len(dsG))
size_learn = len(dsG) - size_test

In [77]:
dsG_train, dsG_test = torch.utils.data.random_split(dsG, [size_learn, size_test])

In [78]:
dlr_train = PyG.loader.DataLoader(dsG_train, batch_size=64, shuffle=True)

## Below blcok is used to check the data operation correctness

In [76]:
dlr = PyG.loader.DataLoader(dsG, batch_size=64, shuffle=False)

k = next(iter(dlr))
x = k.x

# x.shape
# x.view(64, -1, 12).shape

x2 = x.view(64, -1, 12).permute(2,0,1)
# x2.shape
# x2[:,0,:]

df = pd.read_csv('data/sz_speed.csv')
# df.iloc[:12, :].values
(x2[:,0,:] - torch.Tensor(df.iloc[:12, :].values)).max()

tensor(0.)

+ **GCN operator:**

$\mathbf{X}^{\prime} = \mathbf{\hat{D}}^{-1/2} \mathbf{\hat{A}}
\mathbf{\hat{D}}^{-1/2} \mathbf{X} \mathbf{\Theta},$

+ **GRU operator:**

$r= \sigma ( W_ {ir} x + b_ {ir} + W_ {hr} h+ b_ {hr} )$

$z= \sigma  (  W_ {iz}  x+  b_ {iz}  +  W_ {hz}  h+  b_ {hz}  )$

$n=  \tanh  (  W_ {in}  x+  b_ {in}  +r*(  W_ {hn}  h+  b_ {hn}  ))$

$h'=(1-z)*n+z*h$





+ 一种思路(T-GCN, GCN1GRU), 把GCN嵌入到GRU里去, 即在$W_ir, W_hr$等weight前都乘上一个$\hat{A}_{DD}$
+ 另一种思路(GCN2GRU), 把GCN传给GRU, 即GRU公式里的x是GCN得到的$X^{'}$
+ 都试试吧

## GCN2GRU

+ 我靠, 不对啊, 有个问题, 我GCN完了之后, sequence就没了啊, 因为我变成了额, 一个hidden dim的feature, 他不再是那什么了?
+ 但是换个思路, 如果把12个也当成图, 我只要hidden dim是12的倍数, 然后拆出来得到seq就好?比如36, 那我理解为每个seq变成3个attr, 再和nnodes相乘, 变成seq, nnodes*nfeatures这样? 似乎也说得过去...试试

In [ ]:
class GCN2GRU(torch.nn.Module):  # 
    def __init__(self, input_dim, hidden_dim_GCN, hidden_dim_GRU, output_dim):
        super().__init__()
        torch.manual_seed(925)
        self.conv1 = GCNConv(input_dim, hidden_dim) # 只需定义好输入特征和输出特征即可
        self.gru = torch.nn.GRU(hidden_dim_GCN/input_dim*156, 156*64)
        self.regressor = Linear(156*64, 156*3)

    def forward(self, x, edge_index):
        # x : (batch_size, nnodes, nFeatrues)
        
        h = self.conv1(x, edge_index) # h: (batch_size, nNodes, n_hiddenG)
        h = h.tanh()
        
        # 分类层
        out = self.regressor(h)

        return out

In [23]:
ds = TGCN_dataset()

Processing ... ...
Process done.
